In [1]:
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

In [11]:
def make_url(x_url):
    response = requests.get(x_url, headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0 x64)'})
    text = response.text
    html = BeautifulSoup(text, 'html.parser')
    return html
    
def find_page(x_page_number,x_url,x_html):
    target_text = "공략 더 보기"
    target_element = x_html.find(string=target_text)
    parent_element = target_element.find_parent().parent.parent.get("class")
    x_page = x_url + x_html.find_all('div',{'class':parent_element})[x_page_number].find('a')['href'] #class = css-1jardaz emls75t6
    return x_page

def find_table():
    LV5_button = driver.find_element(By.XPATH, "//*[@id=\"content-container\"]/div[2]/div[2]/div[1]/div[1]/div[2]/div/nav/div[1]")
    LV5_button.click()
    sleep(0.2)
    x_target_table = driver.find_element(By.XPATH,'//*[@id="content-container"]/div[2]/div[2]/div[1]/div[1]/div[1]')
    return x_target_table

def first_augmenter():
    x_target_deck_name = driver.find_element(By.XPATH,'//*[@id="content-container"]/div[2]/div[2]/div[2]/div/div[1]/div/div/div[1]/h2')
    return x_target_deck_name


def find_deck_name(x_html,x_deck_name_list):
    x_deck_name = x_html.find('div',{'class':'css-1s5hngw e691syl3'}).find('h2').text
    x_deck_name_list.append(x_deck_name)
    return x_deck_name_list

def find_item(x_html,x_deck_name_list):
    try:
        t = x_html.find('div',{'class':'css-1s5hngw e691syl3'}).text
        x_deck_name_list.append(t.split('수집 아이템 추천')[1].split('활용 아이템')[0].replace('\xa0',''))
    except:
        x_deck_name_list.append("")
    return x_deck_name_list

## 덱 이름이랑 아이템 불러오기

In [12]:
url = 'https://lolchess.gg'


html = make_url(url+'/meta')

deck_name_list = []
item_list = []
list_names = [deck_name_list, item_list]
function_names2 = [find_deck_name, find_item]

number = len(html.find_all('div',{'class':'css-128625v emls75t7'}))
numbers = range(1,number)

for i in numbers:
    for function_name2, list_name in zip(function_names2, list_names):
        page = find_page(i, url, html)
        tip_page = make_url(page) #html.paser
        
        function_name2(tip_page, list_name)

In [13]:
url = 'https://lolchess.gg'
html = make_url(url+'/meta')

numbers = range(1,len(html.find_all('div',{'class':'css-128625v emls75t7'})))

tag_list = []

for i in numbers:
    if "HOT" in html.find_all('div',{'class':'css-1xsl2fm emls75t4'})[i].text:
        a = html.find_all('div',{'class':'css-1xsl2fm emls75t4'})[i].find('span',{'class':'tag hot'}).text
        tag_list.append(a)
        
    elif "UPDATED" in html.find_all('div',{'class':'css-1xsl2fm emls75t4'})[i].text:
        a = html.find_all('div',{'class':'css-1xsl2fm emls75t4'})[i].find('span',{'class':'tag updated'}).text
        tag_list.append(a)
        
    elif "NEW" in html.find_all('div',{'class':'css-1xsl2fm emls75t4'})[i].text:
        a = html.find_all('div',{'class':'css-1xsl2fm emls75t4'})[i].find('span',{'class':'tag new'}).text
        tag_list.append(a)
        
    else:
        a = ""
        tag_list.append(a)

In [14]:
link1_list =[]
table_image_list = []
deck_image_list = []

for i in numbers:
    link1_list.append(find_page(i, url, html))
    table_image_list.append(f'table/table_{i}.png')
    deck_image_list.append(f'deck/deck_{i}.png')


print(len(link1_list), len(table_image_list), len(deck_image_list))

30 30 30


### html body 태그 생성기

In [15]:
text_list = ""

for link1, deck_name, table_image, deck_image, item, tag in zip(link1_list, deck_name_list, table_image_list, deck_image_list, item_list, tag_list):
    if 'B.F.대검' in item:
        item = item.replace('B.F.대검','BF 대검')
        item = re.sub(r'\.', '.<br>', item)
        
    elif 'B.F. 대검' in item:
        item = item.replace('B.F. 대검','BF 대검')
        item = re.sub(r'\.', '.<br>', item)
    else:
        item = re.sub(r'\.', '.<br>', item)
    
    text = f"""<h2><a href="{link1}">{deck_name}</a>{tag}</h2><img src="{table_image}"><br><img src="{deck_image}"><br><div class="text-content">{item}</div><hr>"""
    text_list += text

In [16]:
import clipboard

clipboard.copy(text_list)